# Exploratory data analysis

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
data  = pd.read_csv('data/D1.csv', low_memory=False)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50031 entries, 0 to 50030
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              50031 non-null  int64 
 1   patient_nbr               50031 non-null  int64 
 2   race                      50031 non-null  object
 3   gender                    50031 non-null  object
 4   age                       50031 non-null  object
 5   weight                    50031 non-null  object
 6   admission_type_id         50031 non-null  int64 
 7   discharge_disposition_id  50031 non-null  int64 
 8   admission_source_id       50031 non-null  int64 
 9   length_of_stay            50031 non-null  int64 
 10  payer_code                50031 non-null  object
 11  medical_specialty         50031 non-null  object
 12  num_lab_procedures        50031 non-null  int64 
 13  num_procedures            50031 non-null  int64 
 14  num_medications       

In [4]:
def value_counts(df):
    """
    Count the occurrences of each unique value in the object columns of a DataFrame.
    Present proportions of each unique value.

    :param df: A pandas DataFrame object.
    :return: None
    """
    for column in df.columns:
        if df[column].dtype == 'object':
            print(df[column].value_counts(normalize=True))
            print('-' * 50)


In [5]:
# value_counts(data)

In [6]:
data[['number_outpatient', 'number_inpatient', 'number_emergency']]

,number_outpatient,number_inpatient,number_emergency
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
50026,0,0,0
50027,1,0,0
50028,0,0,0
50029,0,0,0


In [7]:
data['number_inpatient'].value_counts(normalize=True)

number_inpatient
0     0.684975
1     0.184386
2     0.068518
3     0.030141
4     0.014291
5     0.007136
6     0.004237
7     0.002419
8     0.001299
9     0.000859
10    0.000520
?     0.000300
11    0.000300
12    0.000220
13    0.000100
15    0.000080
14    0.000080
16    0.000080
17    0.000020
21    0.000020
18    0.000020
Name: proportion, dtype: float64

In [8]:
data['number_outpatient'].value_counts(normalize=True)

number_outpatient
0     0.888589
1     0.062501
2     0.021806
3     0.012732
4     0.006696
5     0.003538
6     0.001339
7     0.000620
8     0.000560
?     0.000400
9     0.000320
10    0.000240
11    0.000180
12    0.000100
13    0.000080
14    0.000080
16    0.000060
15    0.000040
20    0.000020
21    0.000020
35    0.000020
17    0.000020
29    0.000020
36    0.000020
Name: proportion, dtype: float64

In [9]:
data['number_emergency'].value_counts(normalize=True)

number_emergency
0     0.923128
1     0.052547
2     0.012872
3     0.004637
4     0.002638
?     0.001379
5     0.000899
6     0.000540
7     0.000480
8     0.000320
9     0.000200
10    0.000160
11    0.000060
22    0.000040
25    0.000020
13    0.000020
42    0.000020
16    0.000020
28    0.000020
Name: proportion, dtype: float64

In [10]:
data['weight'].value_counts(normalize=True)

weight
?            0.962783
[75-100)     0.015870
[50-75)      0.010993
[100-125)    0.006316
[125-150)    0.001459
[25-50)      0.001359
[0-25)       0.000740
[150-175)    0.000340
[175-200)    0.000120
>200         0.000020
Name: proportion, dtype: float64

## Comments about the data types

- The `id` columns are integers, which is fine.
- The `race` and `gender` look fine, just some missing values. We should convert them to categorical data type.
- The `age` column is a string based on the formatting of the intervals. We should convert it to an interval data type.
- The `weight` column has 97% missing values. I suggest drop this column.
- The `payer_code` column has 40% missing values. We should discuss if it is necessary to keep this column. We might be able to assume that the emply payer code means that the patient does not have insurance. If not, I suggest drop this column.
- 
- 

## Comments about the goal of the data mining

This looks like a 'length of stay' prediction problem. The goal is to predict the length of stay of a patient in the hospital. The `length_of_stay` column is the target variable. It has no missing values and the data are in a manageable range. We should convert this column to a numeric data type. 

The `readmitted` column could be secondary target variable. It is a categorical variable with three classes. We should convert this column to a categorical data type.

The `discharge_disposition_id` could also be used as a secondary target variable. It is a categorical variable with 26 classes. It might be worth reducing the number of classes to binary outcome variable (all cause mortality), or categorical variable with fewer classes (e.g. discharged home, discharged to another facility, died.).

We should discuss if we want to filter out the `admission_type_id` column. If we choose length of stay as the target variable, we might want to filter out the `admission_type_id` column to exclude newborns and electives. The same goes for `single_day_admission`. We might want to filter out the single day admissions. 

In [11]:
data['length_of_stay'].value_counts(normalize=True)

length_of_stay
3     0.166677
2     0.164018
4     0.134497
1     0.133717
5     0.097220
6     0.076233
7     0.058764
8     0.047231
9     0.032060
10    0.026364
11    0.020947
12    0.016770
13    0.013252
14    0.012252
Name: proportion, dtype: float64

In [12]:
data['admission_type_id'].value_counts(normalize=True)

admission_type_id
1    0.484999
2    0.194180
3    0.167816
6    0.083628
5    0.066099
8    0.003098
4    0.000140
7    0.000040
Name: proportion, dtype: float64

### Data mismatch corrections

Some of the data fields have a `?` character. We should replace these with `np.nan` and convert the columns to correct data type.

In [13]:
# Make a copy for data quality corrections

data_quality = data.copy()

In [14]:
data.gender.value_counts()

gender
Female             27000
Male               23030
Unknown/Invalid        1
Name: count, dtype: int64

In [15]:
# Replace `?` characters in the data['race'] column with NA
data_quality.loc[:,'race'] = data['race'].replace('?', 'Other')

In [16]:
data['race'].value_counts()

race
Caucasian          35732
AfricanAmerican    11149
?                   1257
Hispanic            1020
Other                610
Asian                263
Name: count, dtype: int64

In [17]:
data_quality.race.value_counts()

race
Caucasian          35732
AfricanAmerican    11149
Other               1867
Hispanic            1020
Asian                263
Name: count, dtype: int64

In [18]:
data_quality.gender.value_counts()

gender
Female             27000
Male               23030
Unknown/Invalid        1
Name: count, dtype: int64

In [19]:
data.loc[0:6, 'race']

0          Caucasian
1          Caucasian
2          Caucasian
3    AfricanAmerican
4          Caucasian
5    AfricanAmerican
6          Caucasian
Name: race, dtype: object

In [20]:
data_quality = data[data['gender'] != 'Unknown/Invalid']

In [21]:
data['age'].value_counts()

age
[70-80)     13109
[60-70)     10874
[50-60)      8775
[80-90)      7530
[40-50)      5064
[30-40)      2053
[90-100)     1178
[20-30)       842
[10-20)       468
[0-10)        138
Name: count, dtype: int64

In [22]:
data['weight'].value_counts(normalize=True)

weight
?            0.962783
[75-100)     0.015870
[50-75)      0.010993
[100-125)    0.006316
[125-150)    0.001459
[25-50)      0.001359
[0-25)       0.000740
[150-175)    0.000340
[175-200)    0.000120
>200         0.000020
Name: proportion, dtype: float64

In [23]:
data['discharge_disposition_id'].value_counts()

discharge_disposition_id
1     29385
3      5856
6      5309
18     3668
2       981
25      975
5       952
11      847
22      704
4       486
7       268
23      181
14      137
13       99
8        98
15       21
28       19
17       14
16       11
10        6
24        4
12        3
9         3
20        2
19        1
27        1
Name: count, dtype: int64

In [24]:
data['payer_code'].value_counts(normalize=True)

payer_code
?     0.652895
MC    0.187983
HM    0.031441
BC    0.029822
UN    0.025104
SP    0.023985
MD    0.019968
CP    0.014651
CM    0.004817
OG    0.003078
DM    0.002738
PO    0.001619
WC    0.000700
SI    0.000580
CH    0.000380
OT    0.000240
Name: proportion, dtype: float64

In [25]:
data_quality.loc[:,'payer_code'] = data.loc[:, 'payer_code'].replace('?', 'Unknown')

In [26]:
data_quality['payer_code'].value_counts(normalize=True)

payer_code
Unknown    0.652888
MC         0.187987
HM         0.031441
BC         0.029822
UN         0.025105
SP         0.023986
MD         0.019968
CP         0.014651
CM         0.004817
OG         0.003078
DM         0.002738
PO         0.001619
WC         0.000700
SI         0.000580
CH         0.000380
OT         0.000240
Name: proportion, dtype: float64

In [27]:
data.medical_specialty.value_counts(normalize=True)

medical_specialty
?                                0.354880
InternalMedicine                 0.215606
Family/GeneralPractice           0.109812
Cardiology                       0.073674
Emergency/Trauma                 0.039076
                                   ...   
SurgicalSpecialty                0.000020
Proctology                       0.000020
Psychiatry-Addictive             0.000020
Pediatrics-InfectiousDiseases    0.000020
Cardiology-Pediatric             0.000020
Name: proportion, Length: 68, dtype: float64

In [28]:
data_quality.loc[:,'medical_specialty'] = data.loc[:, 'medical_specialty'].replace('?', 'Unknown')

In [29]:
data_quality['medical_specialty'].value_counts(normalize=True)

medical_specialty
Unknown                          0.354887
InternalMedicine                 0.215611
Family/GeneralPractice           0.109814
Cardiology                       0.073656
Emergency/Trauma                 0.039077
                                   ...   
SurgicalSpecialty                0.000020
Proctology                       0.000020
Psychiatry-Addictive             0.000020
Pediatrics-InfectiousDiseases    0.000020
Cardiology-Pediatric             0.000020
Name: proportion, Length: 68, dtype: float64

In [36]:
data_quality['number_inpatient'].value_counts()


number_inpatient
0     34269
1      9225
2      3428
3      1508
4       715
5       357
6       212
7       121
8        65
9        43
10       26
?        15
11       15
12       11
13        5
15        4
14        4
16        4
17        1
21        1
18        1
Name: count, dtype: int64

In [40]:
data_quality['diag_3'].value_counts()

diag_3
250    6234
401    4429
428    2210
276    2208
427    1941
       ... 
308       1
735       1
697       1
341       1
868       1
Name: count, Length: 671, dtype: int64

In [42]:
data_quality['number_diagnoses'].value_counts()
data_quality['number_diagnoses'].dtype

dtype('int64')

In [43]:
data_quality['diabetes'].value_counts()

diabetes
Yes    37368
No     12662
Name: count, dtype: int64

In [45]:
data_quality['max_glu_serum'].value_counts(normalize=True)

max_glu_serum
Norm    0.507868
>200    0.295969
>300    0.196163
Name: proportion, dtype: float64

In [46]:
data_quality['A1Cresult'].value_counts()

A1Cresult
>8      4360
Norm    1692
>7      1561
Name: count, dtype: int64

In [47]:
data_quality['metformin'].value_counts()

metformin
No        40950
Steady     8224
Up          572
Down        284
Name: count, dtype: int64

In [48]:
data_quality['repaglinide'].value_counts()

repaglinide
No        49436
Steady      522
Up           48
Down         24
Name: count, dtype: int64

In [49]:
data_quality['nateglinide'].value_counts()

nateglinide
No        49768
Steady      254
Up            5
Down          3
Name: count, dtype: int64

In [50]:
data_quality['chlorpropamide'].value_counts()

chlorpropamide
No        49962
Steady       61
Up            5
Down          1
?             1
Name: count, dtype: int64

In [51]:
data_quality['glimepiride'].value_counts()

glimepiride
No        47776
Steady     1989
Up          175
Down         90
Name: count, dtype: int64

In [52]:
data_quality['acetohexamide'].value_counts()

acetohexamide
No    50030
Name: count, dtype: int64

In [54]:
data_quality['glipizide'].value_counts()

glipizide
No        43570
Steady     5685
Up          469
Down        306
Name: count, dtype: int64

In [56]:
data_quality['glyburide'].value_counts()    

glyburide
No        43835
Steady     5318
Up          527
Down        350
Name: count, dtype: int64

In [57]:
data_quality['tolbutamide'].value_counts()

tolbutamide
No    50030
Name: count, dtype: int64

In [58]:
data_quality['insulin'].value_counts()

insulin
No        25753
Steady    15992
Down       4587
Up         3698
Name: count, dtype: int64

In [59]:
data_quality['change'].value_counts()

change
No    29669
Ch    20361
Name: count, dtype: int64

In [60]:
data_quality['diabetesMed'].value_counts()

diabetesMed
Yes    37401
No     12629
Name: count, dtype: int64

In [61]:
data_quality['readmitted'].value_counts()

readmitted
NO     26940
>30    17341
<30     5749
Name: count, dtype: int64

In [62]:
data_quality['single_day_admission'].value_counts()

single_day_admission
No     43341
Yes     6689
Name: count, dtype: int64